# Preprocessing - Session Logs

In [1]:
#Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
from more_itertools import unique_everseen

In [2]:
#Data imports
PATH = "../../data/new_data/"
#sessions_file = (PATH+'sis-log.json')
sessions_file = (PATH+'clustinator-input.json')

In [58]:
df = pd.read_json(sessions_file)
df.head()

,app-id,version,tailoring,epsilon,min-sample-size,start-micros,interval-start-micros,end-micros,sessions
0,app.service,v1.0,all,1.5,10,1564714800000,1564729200000,1564732800000,{'unique-id': '8687d9c93f7e3ad7615b8522b2fee6c...
1,app.service,v1.0,all,1.5,10,1564714800000,1564729200000,1564732800000,{'unique-id': '731bb951faef1428c778d1dd25a18dd...
2,app.service,v1.0,all,1.5,10,1564714800000,1564729200000,1564732800000,{'unique-id': '34fa36453caf4be276b867dc782e3a7...
3,app.service,v1.0,all,1.5,10,1564714800000,1564729200000,1564732800000,{'unique-id': 'f02471c9630c01c4fe7495b6cb29566...
4,app.service,v1.0,all,1.5,10,1564714800000,1564729200000,1564732800000,{'unique-id': 'df1a6e52b93bb29432f22593142226f...


In [59]:
df[['app-id','version', 'tailoring',  'min-sample-size', 'start-micros',
    'interval-start-micros', 'end-micros']].to_dict('r')[0]

{'app-id': 'app.service',
 'version': 'v1.0',
 'tailoring': 'all',
 'min-sample-size': 10,
 'start-micros': 1564714800000,
 'interval-start-micros': 1564729200000,
 'end-micros': 1564732800000}

In [60]:
df = pd.read_json(sessions_file)
print(df.head())
epsilon = df.loc[[0], ['epsilon']].get_values()[0][0]
min_samples = df.loc[[0], ['min-sample-size']].get_values()[0][0]
start = df.loc[[0], ['start-micros']].get_values()[0][0]
step_size = df.loc[[0], ["interval-start-micros"]].get_values()[0][0]
stop = df.loc[[0], ["end-micros"]].get_values()[0][0]

print(type(start),stop)

        app-id version tailoring  epsilon  min-sample-size   start-micros  \
0  app.service    v1.0       all      1.5               10  1564714800000   
1  app.service    v1.0       all      1.5               10  1564714800000   
2  app.service    v1.0       all      1.5               10  1564714800000   
3  app.service    v1.0       all      1.5               10  1564714800000   
4  app.service    v1.0       all      1.5               10  1564714800000   

   interval-start-micros     end-micros  \
0          1564729200000  1564732800000   
1          1564729200000  1564732800000   
2          1564729200000  1564732800000   
3          1564729200000  1564732800000   
4          1564729200000  1564732800000   

                                            sessions  
0  {'unique-id': '8687d9c93f7e3ad7615b8522b2fee6c...  
1  {'unique-id': '731bb951faef1428c778d1dd25a18dd...  
2  {'unique-id': '34fa36453caf4be276b867dc782e3a7...  
3  {'unique-id': 'f02471c9630c01c4fe7495b6cb29566...  
4  

In [67]:
s_r_dict = {}
states = []

for sessions in df['sessions']:
    #print(sessions)
    tmp_list = []
    #print(sessions['unique-id'],'\n')
    key = sessions['unique-id']
    #print(sessions['requests'][0]['endpoint'])
    value = sessions['requests'][0]['endpoint']
    for value in sessions['requests']:
        tmp_list.append(value['endpoint'])
        states.append(value['endpoint'])
    s_r_dict[key] = tmp_list

if len(states) > len(tmp_list):
    states = list(unique_everseen(states+tmp_list))
else:
    states = list(unique_everseen(tmp_list+states))

states.insert(0,'INITIAL*')
states.append('$')
#print(states, '\n')
print(s_r_dict)

{'8687d9c93f7e3ad7615b8522b2fee6c05d3109c96d2177bbe23d1ebeb374d6b9_1535679033000000': ['courseCatalogueUsingGET'], '731bb951faef1428c778d1dd25a18dd9b4057ca1194d30578641a30104ada7e2_1535679670000000': ['courseCatalogueUsingGET'], '34fa36453caf4be276b867dc782e3a7d2a4372d8b75ee2fbd109f382db8ae5a5_1535679319000000': ['courseCatalogueUsingGET'], 'f02471c9630c01c4fe7495b6cb295666dd0fe0539a5a16cb81d82a98c10d7d17_1535679540000000': ['overviewUsingGET', 'credentialsVerificationUsingPOST', 'examResultUsingGET', 'examTermBrowserUsingGET', 'examTermBrowserUsingGET', 'examTermBrowserUsingGET', 'examTermBrowserUsingGET', 'examTermBrowserUsingGET', 'examTermBrowserUsingGET'], 'df1a6e52b93bb29432f22593142226f67c23a420c4084eba663005bc647bfb54_1535680139000000': ['credentialsVerificationUsingPOST', 'loginUsingGET', 'credentialsVerificationUsingPOST', 'credentialsVerificationUsingPOST', 'loginUsingGET'], '87c5fe61887eee08435f278018bbef2dff317c341bc1ed733732b759feb8300e_1535678406000000': ['codeListUsingG

---

In [3]:
with open(sessions_file) as f:
    data = json.load(f)

header_dict = {}
header_list = ['app-id','version', 'tailoring',  'min-sample-size', 'start-micros',
    'interval-start-micros', 'end-micros']
for header in header_list:
    for k, v in data.items():
        if k == header:
            header_dict[k]=v
            #print(k, v)
print(header_dict)            

{'app-id': 'sis', 'version': 'v1', 'tailoring': 'all', 'min-sample-size': 10, 'start-micros': 1535657400000000, 'interval-start-micros': 1535675400000000, 'end-micros': 1535679000000000}


In [4]:
for k, v in data.items():
       print(k)

app-id
version
tailoring
epsilon
min-sample-size
start-micros
interval-start-micros
end-micros
states
previous-markov-chains
sessions


In [5]:
cluster_param = ['epsilon', 'min-sample-size']
params = []
for param in cluster_param:
    params.append(data[param])
print(params)

[1.5, 10]


In [10]:
len(data['states'])

34

In [9]:
s_r_dict = {}
states = []

for sessions in data['sessions']:
    #print(sessions)
    tmp_list = []
    #print(sessions['unique-id'],'\n')
    key = sessions['unique-id']
    #print(sessions['requests'][0]['endpoint'])
    value = sessions['requests'][0]['endpoint']
    for value in sessions['requests']:
        tmp_list.append(value['endpoint'])
        states.append(value['endpoint'])
    s_r_dict[key] = tmp_list

if len(states) > len(tmp_list):
    states = list(unique_everseen(states+tmp_list))
else:
    states = list(unique_everseen(tmp_list+states))

states.insert(0,'INITIAL*')
states.append('$')
print(len(states), '\n')
#print(s_r_dict)

36 



https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order

https://pypi.org/project/more-itertools/

---

In [3]:
#Data imports
PATH = "../../data/raw/"
sessions_file = (PATH+'sessions.dat')
#Dict of sessions
with open(sessions_file) as fn:
    sessions_raw = fn.readlines()

## Prepared data format

**1. Prepared the session logs into the format of: [{Session-ID:[request-1,request-2..]},{Session-ID:[request-1,request-2..]}...]**

With this format we make sure, that we can identifiy the every single session by the session-id and know all the request steps in this session.

In [4]:
def session_request_dict(sessions_raw):
    dict_list = []
    for session in sessions_raw:
        s_r_dict = {}
        key = re.search('([^.]+)',session).group()
        value = re.findall('\"(.*?)\"',session)
        
        s_r_dict[key]=value
        dict_list.append(s_r_dict)
    
    return dict_list

In [5]:
session_request_dict(sessions_raw)

[{'HZKS0-WG8pZr0eCsZlBAP5Xm': ['login',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'home',
   'logout']},
 {'5tPgZbHdK2Zp+heFBs8HsMkx': ['login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'deferorder',
   'home',
   'logout']},
 {'RU2oCVNdpWEM0-2x7I5OjPbZ': ['login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'purchasecart',
   'home',
   'logout']},
 {'kG4g0E5mqwRYcsQOCfj+7wG7': ['login',
   'inventory',
   'inventory',
   'sellinventory',


**2. Prepared the session logs into the format of: [{Session-ID:[[request-1,request-2],[request-2, request-3],[request-3, request-4]...]},{Session-ID:[[request-1,request-2],[request-2, request-3],[request-3, request-4]...]}...]**

With this format we make sure, that we can identifiy the every single session by the session-id and know the current request and the following request in a list of all request.

In [6]:
def session_passing_info(sessions_raw):
    session_list = []
    
    for session in sessions_raw:
        session_pair_list = []
        s_r_dict = {}
        key = re.search('([^.]+)',session).group()
        value = re.findall('\"(.*?)\"',session)
        
        while len(value) > 1:
            value_pair = value[:2:]
            session_pair_list.append(value_pair)
            value.pop(0)

        s_r_dict[key]=session_pair_list
        session_list.append(s_r_dict)

    
    return session_list

In [7]:
session_passing_info(sessions_raw)

[{'HZKS0-WG8pZr0eCsZlBAP5Xm': [['login', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'View_Items'],
   ['View_Items', 'home'],
   ['home', 'logout']]},
 {'5tPgZbHdK2Zp+heFBs8HsMkx': [['login', 'View_Items_quantity'],
   ['View_Items_quantity', 'Add_to_Cart'],
   ['Add_to_Cart', 'View_Items_quantity'],
   ['View_Items_quantity', 'Add_to_Cart'],
   ['Add_to_Cart', 'View_Items_quantity'],
   ['View_Items_quantity', 'Add_to_Cart'],
   ['Add_to_Cart', 'View_Items_quantity'],
   ['View_Items_quantity', 'Add_to_Cart'],
   ['Add_to_Cart', 'View_Items_quantity'],
   ['View_Items_quantity', 'Add_to_Cart'],
   ['Add_to


**To do: 3. The idea is to prepared the session logs into the format of: [{Session-ID:[{request:[time]},{request:[time]},...]}...]**

With this format we make sure, that we can identifiy the every single session by the session-id and know the current request and the time the costumer takes for this step.

In [ ]:
def session_request_time():
    pass

**4. Prepared the session logs into the format of: {Session-ID:[request-1,request-2..], Session-ID:[request-1,request-2..]...}**

With this format we make sure, that we can identifiy the every single session by the session-id and know all the request steps in this session as a dictionary.

In [5]:
def session_request_dict(sessions_raw):
    s_r_dict = {}
    for session in sessions_raw:

        key = re.search('([^.]+)',session).group()
        value = re.findall('\"(.*?)\"',session)
        
        s_r_dict[key]=value
    
    return s_r_dict
session_request_dict(sessions_raw)

{'HZKS0-WG8pZr0eCsZlBAP5Xm': ['login',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'View_Items',
  'home',
  'logout'],
 '5tPgZbHdK2Zp+heFBs8HsMkx': ['login',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'shoppingcart',
  'remove',
  'deferorder',
  'home',
  'logout'],
 'RU2oCVNdpWEM0-2x7I5OjPbZ': ['login',
  'View_Items_quantity',
  'Add_to_Cart',
  'View_Items_quantity',
  'Add_to_Cart',
  'purchasecart',
  'home',
  'logout'],
 'kG4g0E5mqwRYcsQOCfj+7wG7': ['login',
  'inventory',
  'inventory',
  'sellinventory',
  'sellinventory',
  'sellinventory',
  'home',
  'log

## Time calculating

In [8]:
print(sessions_raw[0:1])

['HZKS0-WG8pZr0eCsZlBAP5Xm.undefined;"login":1434382997367491423:1434382997374959629:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:uid=225969&type=doBrowseVehicles-1&submit=Log+in&action=login:<no-encoding>;"View_Items":1434382999813283029:1434382999822312636:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:category=1&action=View_Items:<no-encoding>;"View_Items":1434383003282325208:1434383003286761252:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:category=1&browse=fwd&action=View_Items:<no-encoding>;"View_Items":1434383004603062394:1434383004612279635:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:category=1&browse=fwd&action=View_Items:<no-encoding>;"View_Items":1434383006059736348:1434383006066448680:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:category=1&browse=fwd&action=View_Items:<no-encoding>;"View_Items":1434383007640162774:1434383007647384242:/specj-web/app:8080:192.168.22.141:HTTP/1.1:GET:category=1&browse=bkwd&action=View_Items:<no-encoding>;"View_Items":1434383009094

In [9]:
from datetime import datetime, timedelta
dt = datetime.fromtimestamp(1434382999813283029 // 1000000000)
print(dt)
dt2 = datetime.fromtimestamp(1434382999822312636 // 1000000000)
tdelta = dt -dt2
print(tdelta)

2015-06-15 17:43:19
0:00:00


https://stackoverflow.com/questions/15649942/python-convert-epoch-time-with-nanoseconds-to-human-readable